In [47]:
import aerospike
import pymongo
from pymongo import MongoClient
import sys

## Configure the clients

The configuration is for 
 - Aerospike database running on port 3000 of localhost (IP 127.0.0.1) which is the default. 
 - Mongo running in a separate container whose IP can be found by `docker inspect <containerid> | grep -i ipaddress`


Modify config if your environment is different (Aerospike database running on a different host or different port).

In [51]:
#Aerospike configuration
aero_config = {
  'hosts': [ ('127.0.0.1', 3000) ]
}
try:
  aero_client = aerospike.client(config).connect()
except:
  print("Failed to connect to the cluster with", config['hosts'])
  sys.exit(1)
print("Connected to Aerospike")

#Mongo configuration
try:
    mongo_client = MongoClient("172.17.0.3", 27017)
    print("Connected to Mongo")
except:
    print("Failed to connect to Mongo")
    sys.exit(1)

Connected to Aerospike
Connected to Mongo


In [101]:
# Store data in Mongo and clear the keys in Aerospike if any
def store_data(data_id, data):
    db = mongo_client["test-database"]
    collection = db['test-collection']
    m_data = {data_id: data}
    collection.drop()
    aero_key = ('test', 'demo', data_id)
    aero_client.remove(aero_key)
    post_id = collection.insert_one(m_data)
store_data("key", "value")

In [104]:
def get_data(data_id, data):
    aero_key = ('test', 'demo', data_id)
    #aero_client.remove(aero_key)
    data_check = aero_client.exists(aero_key)
    if data_check[1]:
        (key, metadata, record) = aero_client.get(aero_key)
        print("Data retrieved from Aerospike cache")
        print("Record::: {} {}".format(data_id, record['value']))
    else:
        mongo_data = collection.find_one({data_id: data})
        print("Data not present in Aerospike cache, retrieved from mongo {}".format(mongo_data))
        aero_client.put(aero_key, {'value': mongo_data[data_id]})
get_data("key", "value")

Data retrieved from Aerospike cache
Record::: key value
